# Example 4: C3Z gate
We extend mixing of two photons on a beam splitter to 8 optical modes,
only two of them interfering. This creates CZ gate operating on two qudits
or C3Z gate operating on the hyperencoded pairs of qubits.

In [16]:
import json
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from json_netlist import instantiate_netlist_components, sp_calculate_effective_matrix
from two_photon_conversion import sp_construct_operator, sp_create_filter_matrix

mode numbering:
* 0 : path 0, polarization H
* 1 : path 0, polarization V
* ...
* 6 : path 3, polarization H
* 7 : path 3, polarization V

it is same for input and output

In [17]:
netlist_json_string = """
{
    "CentralBS_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [2,3,6,7],
      "output_modes" : [2,3,6,7],
      "layer" : 0,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "CentralBS_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,null,null],
      "output_modes" : [0,1,null,null],
      "layer" : 0,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "CentralBS_pt3" : {
      "type" : "BS2pol",
      "input_modes" : [null,null,4,5],
      "output_modes" : [null,null,4,5],
      "layer" : 0,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS1_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,null,null],
      "output_modes" : [0,1,null,null],
      "layer" : 1,
      "arguments" : ["Th1","Tv1"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS1_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [2,3,null,null],
      "output_modes" : [2,3,null,null],
      "layer" : 1,
      "arguments" : ["Th1","Tv1"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS2_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [4,5,null,null],
      "output_modes" : [4,5,null,null],
      "layer" : 1,
      "arguments" : ["Th1","Tv1"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS2_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [6,7,null,null],
      "output_modes" : [6,7,null,null],
      "layer" : 1,
      "arguments" : ["Th1","Tv1"],
      "kw_args" : {"sympy" : true}
    }    
  }
"""
netlist = json.loads(netlist_json_string)

instances = instantiate_netlist_components(netlist)
single_photon_map = sp_calculate_effective_matrix(instances)
single_photon_map #print single photon map


Matrix([
[sqrt(Th)*sqrt(Th1),                  0,                       0,                       0,                  0,                  0,                      0,                      0],
[                 0, sqrt(Tv)*sqrt(Tv1),                       0,                       0,                  0,                  0,                      0,                      0],
[                 0,                  0,      sqrt(Th)*sqrt(Th1),                       0,                  0,                  0, sqrt(Th1)*sqrt(1 - Th),                      0],
[                 0,                  0,                       0,      sqrt(Tv)*sqrt(Tv1),                  0,                  0,                      0, sqrt(Tv1)*sqrt(1 - Tv)],
[                 0,                  0,                       0,                       0, sqrt(Th)*sqrt(Th1),                  0,                      0,                      0],
[                 0,                  0,                       0,                       0, 

how computational basis state are encoded:
* |0000> : signal, idler in modes 0, 4
* |0001> : signal, idler in modes 0, 5
* ...
* |1111> : signal, idler in modes 3, 7

the output encoding is the same.

In [18]:
allowed_inputs = [(i_signal, j_idler+4) for i_signal in range(4) for j_idler in range(4)]
allowed_outputs = [(i_signal, j_idler+4) for i_signal in range(4) for j_idler in range(4)]
n_map_rows = single_photon_map.shape[0]
filter_matrix = sp_create_filter_matrix(allowed_outputs, n_map_rows, True, True)
two_photon_operator = sp_construct_operator(single_photon_map, allowed_inputs, filter_matrix)
#two_photon_operator #print single photon map
sp.N(two_photon_operator.subs([('Th', 1), ('Tv', 1/3), ('Th1', 1/3), ('Tv1', 1)] ),chop=1e-12)

Matrix([
[0.333333333333333,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                  0],
[                0, 0.333333333333333,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                  0],
[                0,                 0, 0.333333333333333,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                 0,                  0],
[                0,                 0,                 0, 0.3333333333